Technological Institute of the Philippines | Quezon City - Computer Engineering
--- | ---
Course Code: | CPE 018
Code Title: | Emerging Technologies in CpE 1 - Fundamentals of Computer Vision
1st Semester | AY 2023-2024
<hr> | <hr>
<u>**ACTIVITY NO.** | **TITLE**
**Name** | LastName, FirstName
**Section** | CPE31Sx
**Date Performed**: |
**Date Submitted**: |
**Instructor**: | Dr. Jonathan V. Taylar / Engr. Verlyn V. Nojor / Engr. Roman M. Richard

<hr>

## 1. Objectives

This activity aims to enable students to perform data preparation and face recognition on their own generated dataset.

## 2. Intended Learning Outcomes (ILOs)
After this activity, the students should be able to:
* Utilize data preparation techniques for images.
* Perform Face Recognition using multiple algorithms.
* Evaluate the performance of different algorithms.

## 3. Procedures and Outputs

### Preparing the training data

Now that we have our data, we need to load these sample pictures into our face recognition algorithms. All face recognition algorithms take two parameters in their `train()` method: an array of images and an array of labels. What do these labels represent? They are the IDs of a certain individual/face so that when face recognition is performed, we not only know the person was recognized but also who—among the many people available in our database—the person is.

To do that, we need to create a comma-separated value (CSV) file, which will contain the path to a sample picture followed by the ID of that person.

**Include a Screenshot of Your Dataset Here**

---

### Loading the data and recognizing faces

In [ ]:
%pip install opencv-python opencv-contrib-python numpy


Next up, we need to load these two resources (the array of images and CSV file) into the face recognition algorithm, so it can be trained to recognize our face. To do this, we build a function that reads the CSV file and—for each line of the file—loads the image at the corresponding path into the images array and the ID into the labels array.

In [10]:
import numpy as np
import os
import cv2

# Load Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Paths
dataset_path = "JOHN/"
model_path = "face_model.yml"
label_dict_path = "label_dictionary.npy"

def read_images(path):
    """ Reads images, detects faces, and prepares training data. """
    X, y = [], []
    label_dict = {}
    label_id = 0  

    for subdirname in os.listdir(path):
        subject_path = os.path.join(path, subdirname)
        if not os.path.isdir(subject_path):
            continue  

        label_dict[label_id] = subdirname  

        for filename in os.listdir(subject_path):
            filepath = os.path.join(subject_path, filename)
            img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)

            if img is None:
                print(f"Skipping {filename}: Invalid image")
                continue  

            faces = face_cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
            if len(faces) == 0:
                print(f"Skipping {filename}: No face detected")
                continue  

            for (x, y_, w, h) in faces:
                face = img[y_:y_ + h, x:x + w]
                face = cv2.resize(face, (200, 200))

                X.append(face)
                y.append(label_id)

        label_id += 1

    return X, y, label_dict

# Read images and extract faces
print("Reading images and training the model...")
X, y, label_dict = read_images(dataset_path)

if len(X) == 0 or len(y) == 0:
    print("Error: No faces found. Check your dataset.")
    exit()

# Convert X and y to correct NumPy array format
X = np.array(X, dtype=np.uint8)  # Convert list of images to NumPy array
y = np.array(y, dtype=np.int32)  # Convert labels to NumPy array

recognizer = cv2.face.EigenFaceRecognizer_create()

# Train the recognizer
recognizer.train(X, y)

# Save trained model and labels
recognizer.save(model_path)
np.save(label_dict_path, label_dict)

print(f"Training complete! Model saved as '{model_path}'. Label dictionary saved as '{label_dict_path}'.")
print("Label dictionary:", label_dict)

Reading images and training the model...
Training complete! Model saved as 'face_model.yml'. Label dictionary saved as 'label_dictionary.npy'.
Label dictionary: {0: 'john', 1: 'naomi'}


**Question: Run the function above on your generated dataset. Provide an analysis and note all the challenges you have encountered running this code.**

---

### Performing Face Recognition Algorithms

Here is a sample script for testing the Face Recognition Algorithm. In this section, we're going to follow the same process but with different algorithms for face recognitions, namely:
- Eigenface Recognition
- Fisherface Recognition
- Local Binary Pattern Histograms (LBPH) Recognition

In [12]:
import numpy as np
import cv2
import os

def read_images(path, face_cascade):
    """ Reads images from the dataset and prepares training data. """
    X, y = [], []
    label_dict = {}
    label_id = 0

    for subdirname in os.listdir(path):
        subject_path = os.path.join(path, subdirname)
        if not os.path.isdir(subject_path):
            continue

        label_dict[label_id] = subdirname

        for filename in os.listdir(subject_path):
            filepath = os.path.join(subject_path, filename)
            img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)

            if img is None:
                continue

            faces = face_cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
            for (x, y_, w, h) in faces:
                face = img[y_:y_ + h, x:x + w]
                face = cv2.resize(face, (200, 200))
                X.append(face)
                y.append(label_id)

        label_id += 1

    return np.array(X, dtype=np.uint8), np.array(y, dtype=np.int32), label_dict  # Return label_dict

def face_rec():
    dataset_path = "JOHN/"  
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    X, y, label_dict = read_images(dataset_path, face_cascade)

    model = cv2.face.EigenFaceRecognizer_create()
    model.train(X, y)

    camera = cv2.VideoCapture(0)

    while True:
        ret, img = camera.read()
        if not ret:
            break

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
            roi = gray[y:y + h, x:x + w]
            roi = cv2.resize(roi, (200, 200), interpolation=cv2.INTER_LINEAR)

            try:
                params = model.predict(roi)
                label = label_dict[params[0]] if params[0] in label_dict else "Unknown"
                print(f"Predicted label: {params[0]}, Confidence: {params[1]:.2f}")  # Debugging information
                cv2.putText(img, f"{label}, {params[1]:.2f}", (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

            except Exception as e:
                print(f"Prediction error: {e}")  # Debugging information
                continue

        cv2.imshow("camera", img)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    camera.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    face_rec()

Predicted label: 0, Confidence: 5647.36
Predicted label: 0, Confidence: 6062.50
Predicted label: 1, Confidence: 9755.50
Predicted label: 0, Confidence: 6062.50
Predicted label: 1, Confidence: 9755.50
Predicted label: 0, Confidence: 6103.70
Predicted label: 0, Confidence: 6104.58
Predicted label: 0, Confidence: 4388.10
Predicted label: 0, Confidence: 4388.10
Predicted label: 0, Confidence: 4701.20
Predicted label: 0, Confidence: 6384.43
Predicted label: 0, Confidence: 6100.79
Predicted label: 0, Confidence: 4534.53
Predicted label: 0, Confidence: 5998.60
Predicted label: 0, Confidence: 6014.52
Predicted label: 0, Confidence: 6288.30
Predicted label: 0, Confidence: 5798.12
Predicted label: 0, Confidence: 5798.12
Predicted label: 0, Confidence: 6265.72
Predicted label: 0, Confidence: 6490.38
Predicted label: 0, Confidence: 6492.73
Predicted label: 0, Confidence: 6557.93
Predicted label: 0, Confidence: 6752.62
Predicted label: 0, Confidence: 6752.62
Predicted label: 0, Confidence: 6737.19


**Question: Provide an analysis of the sample script for the process using the Eigenface Model. What is the sample code doing? Are you able to troubleshoot any problems encountered?**

---
Perform the remaining face recognition techniques by using the same (or modified) process from the sample code:

- `model = cv2.face.createFisherFaceRecognizer()`
- `model = cv2.face.createLBPHFaceRecognizer()`

In [14]:
import numpy as np
import os
import cv2

# Load Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Paths
dataset_path = "JOHN/"
model_path = "face_model_lbph.yml"
label_dict_path = "label_dictionary_lbph.npy"

def read_images(path):
    """ Reads images, detects faces, and prepares training data. """
    X, y = [], []
    label_dict = {}
    label_id = 0  

    for subdirname in os.listdir(path):
        subject_path = os.path.join(path, subdirname)
        if not os.path.isdir(subject_path):
            continue  

        label_dict[label_id] = subdirname  

        for filename in os.listdir(subject_path):
            filepath = os.path.join(subject_path, filename)
            img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)

            if img is None:
                print(f"Skipping {filename}: Invalid image")
                continue  

            faces = face_cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
            if len(faces) == 0:
                print(f"Skipping {filename}: No face detected")
                continue  

            for (x, y_, w, h) in faces:
                face = img[y_:y_ + h, x:x + w]
                face = cv2.resize(face, (200, 200))

                X.append(face)
                y.append(label_id)

        label_id += 1

    return X, y, label_dict

# Read images and extract faces
print("Reading images and training the LBPH model...")
X, y, label_dict = read_images(dataset_path)

if len(X) == 0 or len(y) == 0:
    print("Error: No faces found. Check your dataset.")
    exit()

# Convert X and y to correct NumPy array format
X = np.array(X, dtype=np.uint8)
y = np.array(y, dtype=np.int32)

# Create LBPH recognizer
recognizer = cv2.face.LBPHFaceRecognizer_create()

# Train the recognizer
recognizer.train(X, y)

# Save trained model and labels
recognizer.save(model_path)
np.save(label_dict_path, label_dict)

print(f"Training complete! Model saved as '{model_path}'. Label dictionary saved as '{label_dict_path}'.")
print("Label dictionary:", label_dict)


Reading images and training the LBPH model...
Training complete! Model saved as 'face_model_lbph.yml'. Label dictionary saved as 'label_dictionary_lbph.npy'.
Label dictionary: {0: 'john', 1: 'naomi'}


In [18]:
import numpy as np
import cv2
import os

def read_images(path, face_cascade):
    """ Reads images from the dataset and prepares training data. """
    X, y = [], []
    label_dict = {}
    label_id = 0

    for subdirname in os.listdir(path):
        subject_path = os.path.join(path, subdirname)
        if not os.path.isdir(subject_path):
            continue

        label_dict[label_id] = subdirname

        for filename in os.listdir(subject_path):
            filepath = os.path.join(subject_path, filename)
            img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)

            if img is None:
                continue

            faces = face_cascade.detectMultiScale(img, scaleFactor=1.8, minNeighbors=5, minSize=(30, 30))
            for (x, y_, w, h) in faces:
                face = img[y_:y_ + h, x:x + w]
                face = cv2.resize(face, (200, 200))
                X.append(face)
                y.append(label_id)

        label_id += 1

    return np.array(X, dtype=np.uint8), np.array(y, dtype=np.int32), label_dict  # Return label_dict

def face_rec():
    dataset_path = "JOHN/"  
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    X, y, label_dict = read_images(dataset_path, face_cascade)

    model = cv2.face.EigenFaceRecognizer_create()
    model.train(X, y)

    camera = cv2.VideoCapture(0)

    while True:
        ret, img = camera.read()
        if not ret:
            break

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
            roi = gray[y:y + h, x:x + w]
            roi = cv2.resize(roi, (200, 200), interpolation=cv2.INTER_LINEAR)

            try:
                params = model.predict(roi)
                label = label_dict[params[0]] if params[0] in label_dict else "Unknown"
                print(f"Predicted label: {params[0]}, Confidence: {params[1]:.2f}")  # Debugging information
                cv2.putText(img, f"{label}, {params[1]:.2f}", (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

            except Exception as e:
                print(f"Prediction error: {e}")  # Debugging information
                continue

        cv2.imshow("camera", img)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    camera.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    face_rec()

Predicted label: 1, Confidence: 6492.45
Predicted label: 1, Confidence: 8145.35
Predicted label: 1, Confidence: 6614.13
Predicted label: 1, Confidence: 6417.28
Predicted label: 1, Confidence: 6453.22
Predicted label: 1, Confidence: 7627.99
Predicted label: 1, Confidence: 6453.22
Predicted label: 1, Confidence: 7627.99
Predicted label: 1, Confidence: 6277.42
Predicted label: 1, Confidence: 7638.72
Predicted label: 1, Confidence: 6390.33
Predicted label: 1, Confidence: 6178.21
Predicted label: 1, Confidence: 6128.36
Predicted label: 1, Confidence: 6128.36
Predicted label: 1, Confidence: 6120.37
Predicted label: 1, Confidence: 6250.88
Predicted label: 1, Confidence: 4886.91
Predicted label: 1, Confidence: 4960.64
Predicted label: 1, Confidence: 5393.50
Predicted label: 1, Confidence: 5060.89
Predicted label: 1, Confidence: 4975.49
Predicted label: 1, Confidence: 4928.27
Predicted label: 1, Confidence: 4983.18
Predicted label: 1, Confidence: 4983.18
Predicted label: 1, Confidence: 5743.75


In [27]:
import numpy as np
import os
import cv2

# Load Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Paths
dataset_path = "JOHN/"
model_path = "face_model_fisher.yml"
label_dict_path = "label_dictionary_fisher.npy"

def read_images(path):
    """ Reads images, detects faces, and prepares training data. """
    X, y = [], []
    label_dict = {}
    label_id = 0  

    for subdirname in os.listdir(path):
        subject_path = os.path.join(path, subdirname)
        if not os.path.isdir(subject_path):
            continue  

        label_dict[label_id] = subdirname  

        for filename in os.listdir(subject_path):
            filepath = os.path.join(subject_path, filename)
            img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)

            if img is None:
                print(f"Skipping {filename}: Invalid image")
                continue  

            faces = face_cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
            if len(faces) == 0:
                print(f"Skipping {filename}: No face detected")
                continue  

            for (x, y_, w, h) in faces:
                face = img[y_:y_ + h, x:x + w]
                face = cv2.resize(face, (200, 200))

                X.append(face)
                y.append(label_id)

        label_id += 1

    return X, y, label_dict

# Read images and extract faces
print("Reading images and training the FisherFace model...")
X, y, label_dict = read_images(dataset_path)

if len(X) == 0 or len(y) == 0:
    print("Error: No faces found. Check your dataset.")
    exit()

# Convert X and y to correct NumPy array format
X = np.array(X, dtype=np.uint8)
y = np.array(y, dtype=np.int32)

# Create FisherFace recognizer
recognizer = cv2.face.FisherFaceRecognizer_create()

# Train the recognizer
recognizer.train(X, y)

# Save trained model and labels
recognizer.save(model_path)
np.save(label_dict_path, label_dict)

print(f"Training complete! Model saved as '{model_path}'. Label dictionary saved as '{label_dict_path}'.")
print("Label dictionary:", label_dict)

# Load the trained model and label dictionary for testing
recognizer.read(model_path)
label_dict = np.load(label_dict_path, allow_pickle=True).item()

# Test the recognizer with a sample image
def test_recognizer(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    faces = face_cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    for (x, y, w, h) in faces:
        face = img[y:y + h, x:x + w]
        face = cv2.resize(face, (200, 200))
        label, confidence = recognizer.predict(face)
        print(f"Predicted label: {label_dict[label]}, Confidence: {confidence:.2f}")

# Test with sample images
test_recognizer("path_to_john_image.jpg")
test_recognizer("path_to_naomi_image.jpg")

Reading images and training the FisherFace model...
Skipping 478862282_2704535509737971_2319760447268798302_n.jpg: No face detected
Training complete! Model saved as 'face_model_fisher.yml'. Label dictionary saved as 'label_dictionary_fisher.npy'.
Label dictionary: {0: 'john', 1: 'naomi'}


In [30]:
import numpy as np
import cv2
import os

def read_images(path, face_cascade):
    """ Reads images from the dataset and prepares training data. """
    X, y = [], []
    label_dict = {}
    label_id = 0

    for subdirname in os.listdir(path):
        subject_path = os.path.join(path, subdirname)
        if not os.path.isdir(subject_path):
            continue

        label_dict[label_id] = subdirname

        for filename in os.listdir(subject_path):
            filepath = os.path.join(subject_path, filename)
            img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)

            if img is None:
                continue

            faces = face_cascade.detectMultiScale(img, scaleFactor=1.8, minNeighbors=5, minSize=(30, 30))
            for (x, y_, w, h) in faces:
                face = img[y_:y_ + h, x:x + w]
                face = cv2.resize(face, (200, 200))
                X.append(face)
                y.append(label_id)

        label_id += 1

    return np.array(X, dtype=np.uint8), np.array(y, dtype=np.int32), label_dict  # Return label_dict

def face_rec():
    dataset_path = "JOHN/"  
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    X, y, label_dict = read_images(dataset_path, face_cascade)

    model = cv2.face.EigenFaceRecognizer_create()
    model.train(X, y)

    camera = cv2.VideoCapture(0)

    while True:
        ret, img = camera.read()
        if not ret:
            break

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
            roi = gray[y:y + h, x:x + w]
            roi = cv2.resize(roi, (200, 200), interpolation=cv2.INTER_LINEAR)

            try:
                params = model.predict(roi)
                label = label_dict[params[0]] if params[0] in label_dict else "Unknown"
                print(f"Predicted label: {params[0]}, Confidence: {params[1]:.2f}")  # Debugging information
                cv2.putText(img, f"{label}, {params[1]:.2f}", (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

            except Exception as e:
                print(f"Prediction error: {e}")  # Debugging information
                continue

        cv2.imshow("camera", img)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    camera.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    face_rec()

Predicted label: 0, Confidence: 7799.85
Predicted label: 0, Confidence: 7487.09
Predicted label: 0, Confidence: 8269.83
Predicted label: 0, Confidence: 8087.18
Predicted label: 0, Confidence: 7696.69
Predicted label: 0, Confidence: 8386.73
Predicted label: 1, Confidence: 9108.35
Predicted label: 1, Confidence: 9224.65
Predicted label: 0, Confidence: 9090.80
Predicted label: 1, Confidence: 8978.53
Predicted label: 1, Confidence: 9251.27
Predicted label: 1, Confidence: 9051.09
Predicted label: 1, Confidence: 8942.49
Predicted label: 1, Confidence: 8921.57
Predicted label: 0, Confidence: 8536.98
Predicted label: 1, Confidence: 9252.88
Predicted label: 0, Confidence: 8877.75
Predicted label: 0, Confidence: 8622.65
Predicted label: 0, Confidence: 8684.07
Predicted label: 1, Confidence: 8116.82
Predicted label: 1, Confidence: 7341.62
Predicted label: 1, Confidence: 7551.08
Predicted label: 1, Confidence: 7648.10
Predicted label: 1, Confidence: 7535.31
Predicted label: 1, Confidence: 7609.95


**Question: The `predict()` method returns a two-element array. Provide your analysis of the two returned values and their important ince this application.**

## 4. Supplementary Activity

Your accomplisment of the tasks below contribute to the achievement of ILO1, ILO2, and ILO3 for this module.

---

Tasks:
1. Create a new dataset for testing, this dataset must include the following:
  - The same person/s that the model has to recognize.
  - Different person/s that the model should not recognize.
2. For each model, perform 20 tests. Document the testing performed and provide observations.
3. Conclude on the performed tests by providing your evaluation of the performance of the models.

## 5. Summary, Conclusions and Lessons Learned

<hr/>

***Proprietary Clause***

*Property of the Technological Institute of the Philippines (T.I.P.). No part of the materials made and uploaded in this learning management system by T.I.P. may be copied, photographed, printed, reproduced, shared, transmitted, translated, or reduced to any electronic medium or machine-readable form, in whole or in part, without the prior consent of T.I.P.*